In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
import sklearn.metrics

water_data_m3: pd.DataFrame = pd.read_csv("water_data.csv")
spending_data_quantile_rp: pd.DataFrame = pd.read_csv("spending_data.csv")
population_data: pd.DataFrame = pd.read_csv("population_data.csv")
city_spending_data_rp: pd.DataFrame = pd.read_csv("city_spending_data.csv")
water_spending_rp: pd.DataFrame = pd.read_csv("water_spending.csv")
districts: pd.Series = water_data_m3["District"].to_list()
districts.pop()
display(water_data_m3)
display(population_data)
display(spending_data_quantile_rp)
display(city_spending_data_rp)
display(water_spending_rp)

,District,2017,2018,2019,2020,2021,2022,2023
0,Kedungkandang,6566405,6957957,6364264,7186039,7233227,7472993,7701113
1,Sukun,5864236,6093731,5701541,6088016,6155527,6248290,6546303
2,Klojen,5373575,5469639,4868369,4869511,5162387,5095361,5381262
3,Blimbing,6061895,6308611,6508462,6414178,7147949,6421637,7188327
4,Lowokwaru,7533319,7905237,6872733,7106526,7542021,7899145,8166814
5,Total,31399430,32735175,30315369,31664270,33241111,33137426,34983819


,District,2017,2018,2019,2020,2021,2022,2023
0,Kedungkandang,212459,212459,220055,222742,225337,210211,214157
1,Sukun,209053,209062,214623,216055,216918,202682,204970
2,Klojen,110073,110073,111053,111313,110796,100257,100712
3,Blimbing,198428,198428,202514,203211,203380,189534,190799
4,Lowokwaru,174152,174152,179040,180418,181445,168439,170149
5,Total,904165,904174,927195,933739,937876,871123,880787


,Quantile,2017,2018,2019,2020,2021,2022,2023
0,Q1,507792,601606,558053,546126,566065,698385,648472
1,Q2,790890,906102,878883,851121,905034,995285,950575
2,Q3,1174577,1328382,1318061,1264300,1297978,1349379,1313173
3,Q4,1717888,1870774,1930790,1986977,2035489,1993892,2012753
4,Q5,3914392,3333732,4031312,4178298,4585568,4907348,4851712
5,Average,1619392,1608906,1743569,1766014,1878933,1989831,1954957


,Spending Type,2017,2018,2019,2020,2021,2022,2023
0,Operational,1.473420e+12,1.570000e+12,1.670000e+12,1.690000e+12,1.890000e+12,1.870000e+12,2.190000e+12
1,Assets,3.404940e+11,3.510000e+11,2.390000e+11,3.630000e+11,2.950000e+11,3.110000e+11,4.060000e+11
2,Emergency/Miscellaneous,0.000000e+00,3.408054e+07,1.357371e+09,5.461481e+10,4.555465e+10,6.381986e+09,2.875454e+09
3,Total,1.813920e+12,1.930000e+12,1.910000e+12,2.100000e+12,2.230000e+12,2.190000e+12,2.600000e+12


,District,2017,2018,2019,2020,2021,2022,2023
0,Kedungkandang,34804717300,29254753600,27383762500,30458392400,30845996700,31836763600,41723833400
1,Sukun,31666701400,26482344800,25218673900,26266484900,27125148900,27184392900,36105529800
2,Klojen,35280270500,30587263100,27331348500,26561182200,29455553600,28809525600,36540902000
3,Blimbing,33930849000,28362786600,30878146300,29069956400,34144127000,29165067000,41826587300
4,Lowokwaru,43629441900,38149732000,33386573000,34759242700,36781012200,39114640400,49572070700
5,Total,179311980100,152836880100,144198504200,147115258600,158351838400,156110389500,205768923200


In [23]:
def reshape_to_long(frame: pd.DataFrame, var_name: str, value_name: str) -> pd.DataFrame:
	df = frame.copy(deep=True)
	df.set_index(var_name, inplace=True)
	if "Total" in df.index:
		df.drop("Total", inplace=True)
	if "Average" in df.index:
		df.drop("Average", inplace=True)
	df = df.T
	df.index = df.index.astype(int)
	df.index.name = "Year"
	return df.reset_index().melt(id_vars="Year", var_name=var_name, value_name=value_name)

water_data_m3_long = reshape_to_long(water_data_m3, "District", "Water Used (m3)")
population_data_long = reshape_to_long(population_data, "District", "Population")
city_spending_data_rp_long = reshape_to_long(city_spending_data_rp, "Spending Type", "City Spending (Rp)")
water_spending_rp_long = reshape_to_long(water_spending_rp, "District", "Water Spending (Rp)")
spending_data_quantile_rp_long = reshape_to_long(spending_data_quantile_rp, "Quantile", "Citizen Spending (Rp)")

display(water_data_m3_long)
display(city_spending_data_rp_long)
display(water_spending_rp_long)
display(population_data_long)
display(spending_data_quantile_rp_long)

,Year,District,Water Used (m3)
0,2017,Kedungkandang,6566405
1,2018,Kedungkandang,6957957
2,2019,Kedungkandang,6364264
3,2020,Kedungkandang,7186039
4,2021,Kedungkandang,7233227
5,2022,Kedungkandang,7472993
6,2023,Kedungkandang,7701113
7,2017,Sukun,5864236
8,2018,Sukun,6093731
9,2019,Sukun,5701541


,Year,Spending Type,City Spending (Rp)
0,2017,Operational,1.473420e+12
1,2018,Operational,1.570000e+12
2,2019,Operational,1.670000e+12
3,2020,Operational,1.690000e+12
4,2021,Operational,1.890000e+12
5,2022,Operational,1.870000e+12
6,2023,Operational,2.190000e+12
7,2017,Assets,3.404940e+11
8,2018,Assets,3.510000e+11
9,2019,Assets,2.390000e+11


,Year,District,Water Spending (Rp)
0,2017,Kedungkandang,34804717300
1,2018,Kedungkandang,29254753600
2,2019,Kedungkandang,27383762500
3,2020,Kedungkandang,30458392400
4,2021,Kedungkandang,30845996700
5,2022,Kedungkandang,31836763600
6,2023,Kedungkandang,41723833400
7,2017,Sukun,31666701400
8,2018,Sukun,26482344800
9,2019,Sukun,25218673900


,Year,District,Population
0,2017,Kedungkandang,212459
1,2018,Kedungkandang,212459
2,2019,Kedungkandang,220055
3,2020,Kedungkandang,222742
4,2021,Kedungkandang,225337
5,2022,Kedungkandang,210211
6,2023,Kedungkandang,214157
7,2017,Sukun,209053
8,2018,Sukun,209062
9,2019,Sukun,214623


,Year,Quantile,Citizen Spending (Rp)
0,2017,Q1,507792
1,2018,Q1,601606
2,2019,Q1,558053
3,2020,Q1,546126
4,2021,Q1,566065
5,2022,Q1,698385
6,2023,Q1,648472
7,2017,Q2,790890
8,2018,Q2,906102
9,2019,Q2,878883


In [24]:
city_spending_rp_long_pivot = city_spending_data_rp_long.pivot(index="Year", columns="Spending Type", values="City Spending (Rp)").reset_index()
city_spending_rp_long_pivot.columns = ["Year", "Assets (Rp)", "Emergency/Miscellaneous (Rp)", "Operational (Rp)"]
spending_data_quantile_rp_long_pivot = spending_data_quantile_rp_long.pivot(index="Year", columns="Quantile", values="Citizen Spending (Rp)").reset_index()
spending_data_quantile_rp_long_pivot.columns = ["Year", "Q1 (Rp)", "Q2 (Rp)", "Q3 (Rp)", "Q4 (Rp)", "Q5 (Rp)"]

total_population_yearly = population_data_long.groupby("Year")["Population"].sum().reset_index().rename(columns={"Population":"Total Population"})

df = pd.merge(water_spending_rp_long, population_data_long, on=["Year", "District"])
df = pd.merge(df, water_data_m3_long, on=["Year", "District"])
df = pd.merge(df, city_spending_rp_long_pivot, on="Year", how="left")
df = pd.merge(df, spending_data_quantile_rp_long_pivot, on="Year", how="left")
df = pd.merge(df, total_population_yearly, on="Year", how="left")
df["Water Price / Capita (Rp/m3)"] = (df["Water Spending (Rp)"] / df["Water Used (m3)"])
df["Water Usage / Capita (m3/person)"] = (df["Water Used (m3)"] / df["Population"])
df["Water Spending / Capita (Rp/person)"] = (df["Water Spending (Rp)"] / df["Population"])
df.sort_values(by=["District", "Year"], inplace=True)

from pandas.core.groupby import DataFrameGroupBy

dg: DataFrameGroupBy = df.groupby("District")
parameters: list[str] = ["Water Spending (Rp)", "Water Used (m3)", "Assets (Rp)", "Emergency/Miscellaneous (Rp)", "Operational (Rp)", "Population", "Water Price / Capita (Rp/m3)", "Water Usage / Capita (m3/person)", "Water Spending / Capita (Rp/person)", "Q1 (Rp)", "Q2 (Rp)", "Q3 (Rp)", "Q4 (Rp)", "Q5 (Rp)"]
for parameter in parameters:
	df["Previous " + parameter] = dg[parameter].shift(1)

targets = parameters.copy()
parameters = parameters + ["Previous " + parameter for parameter in parameters]

# Convert invalid values to NaN to detect errors
num_cols: list[str] = [col for col in df.columns if col not in ["Year", "District"]]
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df_model: pd.DataFrame = df.copy().dropna()

del df
del parameter
del col
del dg
del city_spending_rp_long_pivot
del spending_data_quantile_rp_long_pivot
del total_population_yearly

with pd.option_context("display.max_rows", None, "display.max_columns", None):
	display(df_model)

df_model.to_clipboard(sep=",")
df_model.info()

,Year,District,Water Spending (Rp),Population,Water Used (m3),Assets (Rp),Emergency/Miscellaneous (Rp),Operational (Rp),Q1 (Rp),Q2 (Rp),Q3 (Rp),Q4 (Rp),Q5 (Rp),Total Population,Water Price / Capita (Rp/m3),Water Usage / Capita (m3/person),Water Spending / Capita (Rp/person),Previous Water Spending (Rp),Previous Water Used (m3),Previous Assets (Rp),Previous Emergency/Miscellaneous (Rp),Previous Operational (Rp),Previous Population,Previous Water Price / Capita (Rp/m3),Previous Water Usage / Capita (m3/person),Previous Water Spending / Capita (Rp/person),Previous Q1 (Rp),Previous Q2 (Rp),Previous Q3 (Rp),Previous Q4 (Rp),Previous Q5 (Rp)
22,2018,Blimbing,28362786600,198428,6308611,3.510000e+11,3.408054e+07,1.570000e+12,601606,906102,1328382,1870774,3333732,904174,4495.884530,31.792948,142937.421130,3.393085e+10,6061895.0,3.404940e+11,0.000000e+00,1.473420e+12,198428.0,5597.399658,30.549595,170998.291572,507792.0,790890.0,1174577.0,1717888.0,3914392.0
23,2019,Blimbing,30878146300,202514,6508462,2.390000e+11,1.357371e+09,1.670000e+12,558053,878883,1318061,1930790,4031312,927285,4744.307687,32.138331,152474.131665,2.836279e+10,6308611.0,3.510000e+11,3.408054e+07,1.570000e+12,198428.0,4495.884530,31.792948,142937.421130,601606.0,906102.0,1328382.0,1870774.0,3333732.0
24,2020,Blimbing,29069956400,203211,6414178,3.630000e+11,5.461481e+10,1.690000e+12,546126,851121,1264300,1986977,4178298,933739,4532.140580,31.564128,143053.065041,3.087815e+10,6508462.0,2.390000e+11,1.357371e+09,1.670000e+12,202514.0,4744.307687,32.138331,152474.131665,558053.0,878883.0,1318061.0,1930790.0,4031312.0
25,2021,Blimbing,34144127000,203380,7147949,2.950000e+11,4.555465e+10,1.890000e+12,566065,905034,1297978,2035489,4585568,937876,4776.772610,35.145781,167883.405448,2.906996e+10,6414178.0,3.630000e+11,5.461481e+10,1.690000e+12,203211.0,4532.140580,31.564128,143053.065041,546126.0,851121.0,1264300.0,1986977.0,4178298.0
26,2022,Blimbing,29165067000,189534,6421637,3.110000e+11,6.381986e+09,1.870000e+12,698385,995285,1349379,1993892,4907348,871123,4541.687268,33.881188,153877.758080,3.414413e+10,7147949.0,2.950000e+11,4.555465e+10,1.890000e+12,203380.0,4776.772610,35.145781,167883.405448,566065.0,905034.0,1297978.0,2035489.0,4585568.0
27,2023,Blimbing,41826587300,190799,7188327,4.060000e+11,2.875454e+09,2.190000e+12,648472,950575,1313173,2012753,4851712,880787,5818.681774,37.674867,219218.063512,2.916507e+10,6421637.0,3.110000e+11,6.381986e+09,1.870000e+12,189534.0,4541.687268,33.881188,153877.758080,698385.0,995285.0,1349379.0,1993892.0,4907348.0
1,2018,Kedungkandang,29254753600,212459,6957957,3.510000e+11,3.408054e+07,1.570000e+12,601606,906102,1328382,1870774,3333732,904174,4204.503362,32.749646,137695.995933,3.480472e+10,6566405.0,3.404940e+11,0.000000e+00,1.473420e+12,212459.0,5300.421966,30.906693,163818.512278,507792.0,790890.0,1174577.0,1717888.0,3914392.0
2,2019,Kedungkandang,27383762500,220055,6364264,2.390000e+11,1.357371e+09,1.670000e+12,558053,878883,1318061,1930790,4031312,927285,4302.738306,28.921242,124440.537593,2.925475e+10,6957957.0,3.510000e+11,3.408054e+07,1.570000e+12,212459.0,4204.503362,32.749646,137695.995933,601606.0,906102.0,1328382.0,1870774.0,3333732.0
3,2020,Kedungkandang,30458392400,222742,7186039,3.630000e+11,5.461481e+10,1.690000e+12,546126,851121,1264300,1986977,4178298,933739,4238.550946,32.261715,136742.924101,2.738376e+10,6364264.0,2.390000e+11,1.357371e+09,1.670000e+12,220055.0,4302.738306,28.921242,124440.537593,558053.0,878883.0,1318061.0,1930790.0,4031312.0
4,2021,Kedungkandang,30845996700,225337,7233227,2.950000e+11,4.555465e+10,1.890000e+12,566065,905034,1297978,2035489,4585568,937876,4264.486197,32.099597,136888.290427,3.045839e+10,7186039.0,3.630000e+11,5.461481e+10,1.690000e+12,222742.0,4238.550946,32.261715,136742.924101,546126.0,851121.0,1264300.0,1986977.0,4178298.0


<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 22 to 13
Data columns (total 31 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Year                                          30 non-null     int64  
 1   District                                      30 non-null     object 
 2   Water Spending (Rp)                           30 non-null     int64  
 3   Population                                    30 non-null     int64  
 4   Water Used (m3)                               30 non-null     int64  
 5   Assets (Rp)                                   30 non-null     float64
 6   Emergency/Miscellaneous (Rp)                  30 non-null     float64
 7   Operational (Rp)                              30 non-null     float64
 8   Q1 (Rp)                                       30 non-null     int64  
 9   Q2 (Rp)                                       30 non-null     int64  
 

In [25]:
spear_corr = df_model[df_model.columns.difference(["Year", "District", "Water Used (m3)"])].corrwith(df_model["Water Used (m3)"], method="spearman")
display(pd.DataFrame(spear_corr, columns=["Correlation Value"]))
kend_corr = df_model[df_model.columns.difference(["Year", "District", "Water Used (m3)"])].corrwith(df_model["Water Used (m3)"], method="kendall")
display(pd.DataFrame(kend_corr, columns=["Correlation Value"]))
pear_corr = df_model[df_model.columns.difference(["Year", "District", "Water Used (m3)"])].corrwith(df_model["Water Used (m3)"], method="pearson")
display(pd.DataFrame(pear_corr, columns=["Correlation Value"]))

,Correlation Value
Assets (Rp),0.192802
Emergency/Miscellaneous (Rp),0.028187
Operational (Rp),0.271727
Population,0.264961
Previous Assets (Rp),-0.014657
Previous Emergency/Miscellaneous (Rp),0.188292
Previous Operational (Rp),0.219862
Previous Population,0.296239
Previous Q1 (Rp),0.068777
Previous Q2 (Rp),0.068777


,Correlation Value
Assets (Rp),0.146080
Emergency/Miscellaneous (Rp),0.002476
Operational (Rp),0.215407
Population,0.140230
Previous Assets (Rp),0.002476
Previous Emergency/Miscellaneous (Rp),0.146080
Previous Operational (Rp),0.185695
Previous Population,0.163974
Previous Q1 (Rp),0.051995
Previous Q2 (Rp),0.051995


,Correlation Value
Assets (Rp),0.223468
Emergency/Miscellaneous (Rp),-0.037733
Operational (Rp),0.248332
Population,0.572576
Previous Assets (Rp),0.001781
Previous Emergency/Miscellaneous (Rp),0.102302
Previous Operational (Rp),0.193554
Previous Population,0.584257
Previous Q1 (Rp),0.129832
Previous Q2 (Rp),0.114333


In [42]:
districts_map = {}
for i, d in enumerate(districts):
	districts_map[i] = d
	districts_map[d] = i

display(df_model)

,Year,District,Water Spending (Rp),Population,Water Used (m3),Assets (Rp),Emergency/Miscellaneous (Rp),Operational (Rp),Q1 (Rp),Q2 (Rp),...,Previous Operational (Rp),Previous Population,Previous Water Price / Capita (Rp/m3),Previous Water Usage / Capita (m3/person),Previous Water Spending / Capita (Rp/person),Previous Q1 (Rp),Previous Q2 (Rp),Previous Q3 (Rp),Previous Q4 (Rp),Previous Q5 (Rp)
22,2018,Blimbing,28362786600,198428,6308611,3.510000e+11,3.408054e+07,1.570000e+12,601606,906102,...,1.473420e+12,198428.0,5597.399658,30.549595,170998.291572,507792.0,790890.0,1174577.0,1717888.0,3914392.0
23,2019,Blimbing,30878146300,202514,6508462,2.390000e+11,1.357371e+09,1.670000e+12,558053,878883,...,1.570000e+12,198428.0,4495.884530,31.792948,142937.421130,601606.0,906102.0,1328382.0,1870774.0,3333732.0
24,2020,Blimbing,29069956400,203211,6414178,3.630000e+11,5.461481e+10,1.690000e+12,546126,851121,...,1.670000e+12,202514.0,4744.307687,32.138331,152474.131665,558053.0,878883.0,1318061.0,1930790.0,4031312.0
25,2021,Blimbing,34144127000,203380,7147949,2.950000e+11,4.555465e+10,1.890000e+12,566065,905034,...,1.690000e+12,203211.0,4532.140580,31.564128,143053.065041,546126.0,851121.0,1264300.0,1986977.0,4178298.0
26,2022,Blimbing,29165067000,189534,6421637,3.110000e+11,6.381986e+09,1.870000e+12,698385,995285,...,1.890000e+12,203380.0,4776.772610,35.145781,167883.405448,566065.0,905034.0,1297978.0,2035489.0,4585568.0
27,2023,Blimbing,41826587300,190799,7188327,4.060000e+11,2.875454e+09,2.190000e+12,648472,950575,...,1.870000e+12,189534.0,4541.687268,33.881188,153877.758080,698385.0,995285.0,1349379.0,1993892.0,4907348.0
1,2018,Kedungkandang,29254753600,212459,6957957,3.510000e+11,3.408054e+07,1.570000e+12,601606,906102,...,1.473420e+12,212459.0,5300.421966,30.906693,163818.512278,507792.0,790890.0,1174577.0,1717888.0,3914392.0
2,2019,Kedungkandang,27383762500,220055,6364264,2.390000e+11,1.357371e+09,1.670000e+12,558053,878883,...,1.570000e+12,212459.0,4204.503362,32.749646,137695.995933,601606.0,906102.0,1328382.0,1870774.0,3333732.0
3,2020,Kedungkandang,30458392400,222742,7186039,3.630000e+11,5.461481e+10,1.690000e+12,546126,851121,...,1.670000e+12,220055.0,4302.738306,28.921242,124440.537593,558053.0,878883.0,1318061.0,1930790.0,4031312.0
4,2021,Kedungkandang,30845996700,225337,7233227,2.950000e+11,4.555465e+10,1.890000e+12,566065,905034,...,1.690000e+12,222742.0,4238.550946,32.261715,136742.924101,546126.0,851121.0,1264300.0,1986977.0,4178298.0


In [ ]:
from sklearn.svm import SVR
import sklearn.feature_selection
from sklearn.model_selection import GridSearchCV, LeaveOneOut, RepeatedKFold
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing

# Define hyperparameters to tune (example)
param_grid = {
	'C': [0.01, 0.02, 0.05, 0.1, 1],          # Regularization parameter (common to all kernels)
    'epsilon': [0.01, 0.05, 0.1, 0.5],     # Epsilon-SVR parameter (common to all kernels)
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], # The kernel type
    'degree': [2, 3, 4, 5, 6],            # Degree for 'poly' kernel (ignored for others)
    'gamma': ['scale', 'auto', 0.01, 0.05, 0.1, 0.5, 1], # Kernel coefficient for 'rbf', 'poly', 'sigmoid' (ignored for 'linear')
    'coef0': [0.1, 0.5, 0.9, 0.95, 1]
}

scoring = {
    "neg_mse": "neg_mean_squared_error", # Lower is better, negative for maximization
    "neg_rmse": "neg_root_mean_squared_error",
	"neg_mae": "neg_mean_absolute_error",
	"neg_mape": "neg_mean_absolute_percentage_error"
}

# results: dict[str, dict[str, tuple[SVR, dict, float]]] = { district:{ scoring_name:None for scoring_name in scoring.values() } for district in districts }
results: dict[str, tuple[SVR, dict, float]] = {scoring_name:None for scoring_name in scoring.values()}
# for district in districts:
# 	print(f"Measuring best SVR model for district {district}")
for scoring_name, scoring_method in scoring.items():
	svr: SVR = SVR()
	gscv: GridSearchCV = GridSearchCV(svr, param_grid=param_grid, cv=RepeatedKFold(n_splits=5, random_state=69420), scoring=scoring_method, refit=scoring_name, n_jobs=-1)
	# district_df = df_model[df_model["District"] == district].copy()
	X: pd.DataFrame = df_model[["District", "Water Spending (Rp)","Population","Water Used (m3)","Assets (Rp)","Operational (Rp)","Total Population","Water Price / Capita (Rp/m3)", "Water Usage / Capita (m3/person)", "Water Spending / Capita (Rp/person)", "Previous Water Used (m3)", "Previous Water Price / Capita (Rp/m3)", "Previous Water Usage / Capita (m3/person)", "Previous Water Spending / Capita (Rp/person)"]].copy()
	X["District"] = X["District"].map(districts_map)
	Y: pd.DataFrame = pd.DataFrame(df_model["Water Used (m3)"])
	feature_scaler: StandardScaler = StandardScaler()
	target_scaler: StandardScaler = StandardScaler()
	X_norm: np.ndarray = feature_scaler.fit_transform(X)
	Y_norm: np.ndarray = target_scaler.fit_transform(Y.to_numpy().reshape(-1, 1)).ravel()
	gscv.fit(X_norm, Y_norm)
	best_svr = gscv.best_estimator_
	print(f"Best SVR parameters (refit={scoring_name}): {gscv.best_params_}")
	print(f"Best score {gscv.best_score_}")
	results[scoring_name] = {
	# results[district][scoring_name] = {
		"model":best_svr,
		"params":gscv.best_params_,
		"score":gscv.best_score_,
		"feature_scaler": feature_scaler,
		"target_scaler": target_scaler,
		"result_table": gscv.cv_results_
	}
	# Access all scores from cv_results_
	cv_results_df = pd.DataFrame(gscv.cv_results_)
	display(cv_results_df[["param_C","param_coef0","param_degree","param_epsilon","param_gamma","param_kernel","split0_test_score","split1_test_score","split2_test_score","split3_test_score","split4_test_score","split5_test_score", "mean_test_score", "std_test_score", "rank_test_score"]])
	

In [ ]:
from sklearn.neural_network import MLPRegressor

results_mlp = { scoring_name:None for scoring_name in scoring.values() }

mlp_param_list = {
	"hidden_layer_sizes": [(10,20,25),(25,50,70),(30,60,100),(50,100,120)], 
	"activation": ["identity", "logistic", "tanh", "relu"], 
	"solver": ["lbfgs", "sgd", "adam"], 
	"alpha": [0.00001,0.0005,0.0001,0.0005,0.001,0.005,0.05,0.1,0.5]
}

for scoring_name, scoring_method in scoring.items():
	feature_scaler: StandardScaler = StandardScaler()
	target_scaler: StandardScaler = StandardScaler()
	regressor = MLPRegressor()


SVR(C=1, coef0=0.1, degree=2, epsilon=0.01, kernel='linear')
